# Métricas de distancia

En el siguiente cuaderno de Jupyter se ejemplifican las distintas medidas de distancia para la comparación de cadenas.

*Medida de Levenshtein*

La distancia de Levenshtein, también conocida como distancia de edición, es una métrica que se utiliza para medir la diferencia entre dos cadenas de caracteres. Esta distancia mide el número mínimo de operaciones de edición (inserción, eliminación o sustitución) necesarias para convertir una cadena en otra. Es decir, la distancia de Levenshtein calcula cuántos cambios se deben hacer para transformar una cadena en otra. Mientras mayor sea la distancia obtenida, mayor es la diferencia entre ambas cadenas (también llamadas como hileras).

In [2]:
#pip install hermetrics

Note: you may need to restart the kernel to use updated packages.


In [16]:
from hermetrics.levenshtein import Levenshtein

lv = Levenshtein()
word1 = input("Escriba la primera cadena: ")
word2 = input("Escriba la segunda cadena: ")
distancia = lv.distance(word1, word2)
distanciaNormalizada = lv.normalized_distance(word1, word2)
similitud = lv.similarity(word1, word2)

print("La distancia de Levenshtein entre",  word1,  "y",  word2, "es de:", distancia)
print("La distancia normalizada de Levenshtein obtenida es de:", distanciaNormalizada)
print("La similitud de Levenshtein obtenida es de:", similitud)

La distancia de Levenshtein entre cabello y caballero es de: 3
La distancia normalizada de Levenshtein obtenida es de: 0.3333333333333333
La similitud de Levenshtein obtenida es de: 0.6666666666666667


In [17]:
def levenshtein_distance(s, t):
    m = len(s)
    n = len(t)
    d = [[0] * (n + 1) for i in range(m + 1)]
    for i in range(m + 1):
        d[i][0] = i
    for j in range(n + 1):
        d[0][j] = j
    for j in range(1, n + 1):
        for i in range(1, m + 1):
            if s[i-1] == t[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                d[i][j] = min(d[i-1][j] + 1,   # eliminación
                              d[i][j-1] + 1,   # inserción
                              d[i-1][j-1] + 1) # sustitución
    return d[m][n]


def normalized_distance(s, t):
    max_len = max(len(s), len(t))
    if max_len == 0:
        return 0
    else:
        return levenshtein_distance(s, t) / max_len

def similarity(s, t):
    return (1 - normalized_distance(s, t))

word1 = input("Escriba la primera cadena: ")
word2 = input("Escriba la segunda cadena: ")

distanciaX = levenshtein_distance(word1, word2)
distanciaNormalizadaX = normalized_distance(word1, word2)
similitudX = similarity(word1, word2)
print("La distancia de Levenshtein entre",  word1,  "y",  word2, "es de:", distanciaX)
print("La distancia de Levenshtein normalizada es de:", distanciaNormalizadaX)
print("La similitud de Levenshtein obtenida es de:", similitudX)



La distancia de Levenshtein entre cabello y caballero es de: 3
La distancia de Levenshtein normalizada es de: 0.3333333333333333
La similitud de Levenshtein obtenida es de: 0.6666666666666667


# N-Gramas

El algoritmo de N-Gramas es un método para analizar la similitud entre dos cadenas de texto, basado en la ocurrencia de subsecuencias de n caracteres llamadas n-gramas. El algoritmo de N-Gramas utiliza el concepto de bag of words para analizar las similitudes entre las cadenas de texto, es decir, no se tiene en cuenta el orden ni la estructura de las palabras, sino únicamente las palabras individuales que aparecen en cada cadena.

In [41]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def generate_N_grams(text,ngram=1):
  """Genera N-Gramas eliminado las palabras vacías y generando los n-gramas correspondientes."""
  words=[word for word in text.split(" ") if word not in set(stopwords.words('spanish'))]  
  print("Oraciones tras eliminar stopwords: ",words)
  temp=zip(*[words[i:] for i in range(0,ngram)])
  ans=[' '.join(ngram) for ngram in temp]
  return ans

def cos_dist(text1, text2, n):
    from scipy import spatial
    text1 = generate_N_grams(text1, n)
    text2 = generate_N_grams(text2, n)
    print(n,"-gramas de la cadena 1: ", text1)
    print(n,"-gramas de la cadena 2: ", text2)
    full_text = list(set(text1).union(set(text2)))
    print(full_text)
    v1 = [(lambda x: text1.count(x))(x) for x in full_text]
    v2 = [(lambda x: text2.count(x))(x) for x in full_text]
    print(v1, v2)
    return spatial.distance.cosine(v1, v2)


word1 = input("Escriba la primera cadena: ")
word2 = input("Escriba la segunda cadena: ")
n = int(input("Escriba la longitud de los n-gramas: "))

print(word1, word2)

similitudNG = cos_dist(word1, word2, n)
print("La similitud por N-Gramas obtenida es de:", similitudNG)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jorje\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Hola buenos dias como se encuentra usted Hola buenas tardes que tal esta usted
Oraciones tras eliminar stopwords:  ['Hola', 'buenos', 'dias', 'encuentra', 'usted']
Oraciones tras eliminar stopwords:  ['Hola', 'buenas', 'tardes', 'tal', 'usted']
1 -gramas de la cadena 1:  ['Hola', 'buenos', 'dias', 'encuentra', 'usted']
1 -gramas de la cadena 2:  ['Hola', 'buenas', 'tardes', 'tal', 'usted']
['dias', 'Hola', 'buenos', 'encuentra', 'usted', 'buenas', 'tal', 'tardes']
[1, 1, 1, 1, 1, 0, 0, 0] [0, 1, 0, 0, 1, 1, 1, 1]
La similitud por N-Gramas obtenida es de: 0.6


# Jaccard

La distancia de Jaccard es una medida de similitud entre dos conjuntos, y se utiliza para determinar la cantidad de elementos compartidos entre ellos. Se define como el tamaño de la intersección de los conjuntos dividido por el tamaño de la unión de los conjuntos.

In [42]:
from hermetrics.jaccard import Jaccard

jc = Jaccard()
word1 = input("Escriba la primera cadena: ")
word2 = input("Escriba la segunda cadena: ")

distanciaJ = jc.distance(word1, word2)
similitudJ = jc.similarity(word1, word2)

print("La distancia de Jaccard entre",  word1,  "y",  word2, "es de:", distanciaJ)
print("La similitud de Jaccard obtenida es de:", similitudJ)

La distancia de Jaccard entre El hombre que sabe querer y El nombre que no supo querer es de: 0.2666666666666667
La similitud de Jaccard obtenida es de: 0.7333333333333333
